Preliminaries

In [1]:
############### Uncomment the code below for google collab ####################################################
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
#!tar xf spark-3.2.0-bin-hadoop3.2.tgz
#!pip install -q findspark

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [3]:
############### Uncomment the code below for google collab ####################################################
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["SPARK_HOME"] = "spark-3.2.0-bin-hadoop3.2"

In [4]:
import findspark
findspark.init()

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
sc

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/07 15:52:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/07 15:52:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/12/07 15:52:41 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/12/07 15:52:41 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


<SparkContext master=local[*] appName=pyspark-shell>

In [6]:
import pyspark.sql
from pyspark.sql import Row
from pyspark.sql.types import *
import json

In [7]:
import numpy as np
import math
import pandas as pd
np.set_printoptions(precision = 10, suppress = True)

In [8]:
#check the output to see what files still need to be processed
completed = os.listdir('/home/bcavna/Documents/ERA5/processed/smaller/')

finished = []
for c in completed:
    finished.append(c[5:13])
    
checkfiles = list(set(finished))
checkfiles.sort()
print(checkfiles)

['20110101', '20110102', '20110103', '20110104', '20110105', '20110106', '20110107', '20110108', '20110109', '20110110', '20110111', '20110112', '20110113', '20110114', '20110115', '20110116', '20110117', '20110118', '20110119', '20110120', '20110121', '20110122', '20110123', '20110124', '20110125', '20110126', '20110127', '20110128', '20110129', '20110130', '20110201', '20110202', '20110203', '20110204', '20110205', '20110206', '20110207', '20110208', '20110209', '20110210', '20110211', '20110212', '20110213', '20110214', '20110215', '20110216', '20110217', '20110218', '20110219', '20110220', '20110221', '20110222', '20110223', '20110224', '20110225', '20110226', '20110227', '20110228', '20110229', '20110230', '20110301', '20110302', '20110303', '20110304', '20110305', '20110306', '20110307', '20110308', '20110309', '20110310', '20110311', '20110312', '20110313', '20110314', '20110315', '20110316', '20110317', '20110318', '20110319', '20110320', '20110321', '20110322', '20110323', '20

Assign input/output file paths

In [9]:
############### Uncomment the code below for google collab ####################################################
#from google.colab import drive
#drive.mount('/content/drive')

In [10]:
pwd

'/home/bcavna/Documents/ERA5'

In [11]:
cd '/home/bcavna/Documents'

/home/bcavna/Documents


In [12]:
m = 1

In [13]:
y = 2022
y2 = 2021

In [14]:
yearmo = y * 100 + m
print(yearmo)

202201


In [15]:
yearmo2 = y2 * 100 + m
print(yearmo2)

202101


In [16]:
#use this path for google collab : mthly = '/content/drive/My Drive/ERA5/monthly/ERA5_'+ str(yearmo) + '.csv'
mthly = '/home/bcavna/Documents/ERA5/monthly/ERA5_'+ str(yearmo) + '.csv'
print(mthly)

/home/bcavna/Documents/ERA5/monthly/ERA5_202201.csv


In [17]:
output_ws = '/home/bcavna/Documents/ERA5/processed/normvals/WS/WS_FREQ_'+ str(yearmo) + '.csv'
print(output_ws)

/home/bcavna/Documents/ERA5/processed/normvals/WS/WS_FREQ_202201.csv


In [18]:
output_wd = '/home/bcavna/Documents/ERA5/processed/normvals/WD/WD_FREQ_'+ str(yearmo) + '.csv'
print(output_wd)

/home/bcavna/Documents/ERA5/processed/normvals/WD/WD_FREQ_202201.csv


In [19]:
output_vv = '/home/bcavna/Documents/ERA5/processed/normvals/VV/VV_FREQ_'+ str(yearmo) + '.csv'
print(output_vv)

/home/bcavna/Documents/ERA5/processed/normvals/VV/VV_FREQ_202201.csv


In [20]:
mthly2 = '/home/bcavna/Documents/ERA5/ERA5_VV_RH_2021/ERA5_VV_RH_' + str(yearmo2) + '.csv'
print(mthly2)

/home/bcavna/Documents/ERA5/ERA5_VV_RH_2021/ERA5_VV_RH_202101.csv


Get main features file

In [21]:
rawdataDF = spark.read.csv(mthly, header="true")
rawdataDF.createOrReplaceTempView("features")
rawdataDF.show(2)

+-------------------+-----+--------+---------+-------------+---------+--------+---------+
|               time|level|latitude|longitude|            q|        t|       u|        v|
+-------------------+-----+--------+---------+-------------+---------+--------+---------+
|2022-01-01 00:00:00|    1|    42.0|    -89.0|4.1224994e-06|266.58505|41.62961|26.744812|
|2022-01-01 00:00:00|    1|    42.0|   -88.75|4.1224994e-06| 266.6099| 40.8252|27.131168|
+-------------------+-----+--------+---------+-------------+---------+--------+---------+
only showing top 2 rows



Output average wind speed and direction data that will be needed for normalization later

In [22]:
uvDF = spark.sql("SELECT u,v from features")
uvDF.show(5)

+---------+---------+
|        u|        v|
+---------+---------+
| 41.62961|26.744812|
|  40.8252|27.131168|
| 40.02079|27.425846|
|39.263992|27.609201|
|38.559822| 27.67032|
+---------+---------+
only showing top 5 rows



In [23]:
wind_speedR = uvDF.rdd.map(lambda x: round(np.linalg.norm([float(x[0]),float(x[1])]),0)).map(lambda x: float(x)).map(lambda p: Row(ws=p))
wind_speedR.take(2)

[Row(ws=49.0), Row(ws=49.0)]

In [24]:
wind_speedDF = spark.createDataFrame(wind_speedR)
wind_speedDF.createOrReplaceTempView("ws")
wind_speedDF.show(5,truncate=False)

+----+
|ws  |
+----+
|49.0|
|49.0|
|49.0|
|48.0|
|47.0|
+----+
only showing top 5 rows



In [25]:
wsDF = spark.sql("SELECT ws AS WS_SUM, COUNT(ws) AS WS_COUNT from ws GROUP BY ws ORDER BY ws")
#wsDF.show(5)

In [26]:
wsPD = wsDF.toPandas()
wsPD.head()

,WS_SUM,WS_COUNT
0,0.0,7435
1,1.0,70733
2,2.0,161607
3,3.0,220470
4,4.0,245171


In [27]:
wsPD.to_csv(output_ws)

In [28]:
wind_dirR = uvDF.rdd.map(lambda x:[float(x[0]),float(x[1])]).map(lambda x: [x[0],x[1],np.arctan2(x[1],x[0])]).map(lambda x: round(180+float(x[2])*180/math.pi),0).map(lambda p: Row(wd=p))
wind_dirR.take(3)

[Row(wd=213), Row(wd=214), Row(wd=214)]

In [29]:
wind_dirDF = spark.createDataFrame(wind_dirR)
wind_dirDF.createOrReplaceTempView("wd")
wind_dirDF.show(5,truncate=False)

+---+
|wd |
+---+
|213|
|214|
|214|
|215|
|216|
+---+
only showing top 5 rows



In [30]:
#review arctan calculation to understand how it works
check = np.arctan2(-23,7)
check2 = np.linalg.norm((float(-2),float(-23)))
print(check)
print(check2)

-1.2753554896511765
23.08679276123039


In [31]:
wdDF = spark.sql("SELECT wd, COUNT(1) AS WD_COUNT from wd GROUP BY wd ORDER BY wd")
#wdDF.show(500)

In [32]:
wdPD = wdDF.toPandas()
wdPD.head()

,wd,WD_COUNT
0,0,1462
1,1,2923
2,2,2969
3,3,3092
4,4,3195


In [33]:
wdPD.to_csv(output_wd)

Get vertical velocity of the wind (extracted separately)

In [34]:
rawdataDF2 = spark.read.csv(mthly2, header="true")
rawdataDF2.createOrReplaceTempView("features2")
rawdataDF2.show(2)

+-------------------+-----+--------+---------+-------------+------------+
|               time|level|latitude|longitude|            w|           r|
+-------------------+-----+--------+---------+-------------+------------+
|2021-01-01 00:00:00|    1|    42.0|    -89.0|-0.0008459091|0.0004272461|
|2021-01-01 00:00:00|    1|    42.0|   -88.75|-0.0007045269|0.0004272461|
+-------------------+-----+--------+---------+-------------+------------+
only showing top 2 rows



In [35]:
vvDF = spark.sql("SELECT round(w,0) AS vv from features2")
vvDF.createOrReplaceTempView("vv")
vvDF.show(5)

+---+
| vv|
+---+
|0.0|
|0.0|
|0.0|
|0.0|
|0.0|
+---+
only showing top 5 rows



Output average vertical velocity that will be needed for normalization later

In [36]:
vvDF = spark.sql("SELECT vv, COUNT(1) AS VV_COUNT from vv GROUP BY vv ORDER BY vv")
vvDF.show(5)

+----+--------+
|  vv|VV_COUNT|
+----+--------+
|-7.0|       6|
|-6.0|      67|
|-5.0|     312|
|-4.0|     958|
|-3.0|    3972|
+----+--------+
only showing top 5 rows



In [37]:
vvPD = vvDF.toPandas()
vvPD.head()

,vv,VV_COUNT
0,-7.0,6
1,-6.0,67
2,-5.0,312
3,-4.0,958
4,-3.0,3972


In [38]:
vvPD.to_csv(output_vv)

Get relative humidity (extracted separately)

In [39]:
mthly3 = mthly2 #'/home/bcavna/Documents/ERA5/monthly/ERA5_RH/ERA5_RH_'+ str(yearmo) + '.csv'
print(mthly3)

/home/bcavna/Documents/ERA5/ERA5_VV_RH_2021/ERA5_VV_RH_202101.csv


In [40]:
rawdataDF3 = spark.read.csv(mthly3, header="true")
rawdataDF3.createOrReplaceTempView("features3")
rawdataDF3.show(3)

+-------------------+-----+--------+---------+-------------+------------+
|               time|level|latitude|longitude|            w|           r|
+-------------------+-----+--------+---------+-------------+------------+
|2021-01-01 00:00:00|    1|    42.0|    -89.0|-0.0008459091|0.0004272461|
|2021-01-01 00:00:00|    1|    42.0|   -88.75|-0.0007045269|0.0004272461|
|2021-01-01 00:00:00|    1|    42.0|    -88.5|-0.0005633831|0.0004272461|
+-------------------+-----+--------+---------+-------------+------------+
only showing top 3 rows



Combine All Features

In [41]:
allFeatures = spark.sql("SELECT a.time, a.level, a.latitude, a.longitude, a.t, a.u, a.v, round(b.w,0) AS w, b.r FROM features a LEFT JOIN features3 b ON a.time = b.time AND a.level = b.level AND a.latitude = b.latitude AND a.longitude = b.longitude WHERE a.level*1 >=100")
allFeatures.createOrReplaceTempView("allFeatures")
allFeatures.show(10)

+-------------------+-----+--------+---------+---------+----------+----------+---+---------+
|               time|level|latitude|longitude|        t|         u|         v|  w|        r|
+-------------------+-----+--------+---------+---------+----------+----------+---+---------+
|2022-01-01 00:00:00|  100|    42.0|   -88.75| 212.2196| 47.185295| 4.9997272|0.0|4.3478546|
|2022-01-06 20:00:00|  700|   40.25|    -86.5|256.28036| 14.472649|-0.3066671|0.0|19.437172|
|2022-01-07 20:00:00|  100|    42.0|    -86.5|213.55652| 33.455208|-12.216042|0.0|3.5341492|
|2022-01-08 19:00:00|  225|    40.0|   -86.75|220.35072| 31.397821|  8.551147|0.0|30.115097|
|2022-01-13 16:00:00|  200|   38.75|    -88.5|222.05916| 20.504463|-26.626907|0.0|3.8181763|
|2022-01-19 12:00:00|  100|    42.0|   -87.75|216.05186| 27.709068| 0.7257987|0.0|2.9558563|
|2022-01-21 10:00:00|  950|    38.5|    -87.0| 262.2101|-5.4320984|-2.3956099|0.0|60.731293|
|2022-01-22 10:00:00|  100|    42.0|   -88.25| 218.4346| 22.218533| 1.

In [42]:
#check = spark.sql("SELECT COUNT(1) FROM features a JOIN features2 b ON a.time = b.time AND a.level = b.level AND a.latitude = b.latitude" )
#check.take(5)

In [43]:
# all records: 12139848

Assign features to lat/lon cell ids

In [44]:
lonVals = [-89.0, -88.75, -88.5, -88.25, -88.0, -87.75, -87.5, -87.25, -87.0, -86.75, -86.5, -86.25, -86.0, -85.75, -85.5, -85.25, -85.0, -84.75, -84.5, -84.25, -84.0]

In [45]:
latVals = [42.0, 41.75, 41.5, 41.25, 41.0, 40.75, 40.5, 40.25, 40.0, 39.75, 39.5, 39.25, 39.0, 38.75, 38.5, 38.25, 38.0, 37.75, 37.5, 37.25, 37.0]

In [46]:
cells = []

def getCells(lon,lat):
  lonRange = range(len(lon))
  latRange = range(len(lat))
  n = 0
  for i in lonRange:
    for j in latRange:
      cells.append([n,lat[j],lon[i]])
      n = n + 1
  print("total cell count:",n)
  print(cells[:10])
getCells(lonVals,latVals)

total cell count: 441
[[0, 42.0, -89.0], [1, 41.75, -89.0], [2, 41.5, -89.0], [3, 41.25, -89.0], [4, 41.0, -89.0], [5, 40.75, -89.0], [6, 40.5, -89.0], [7, 40.25, -89.0], [8, 40.0, -89.0], [9, 39.75, -89.0]]


In [47]:
cellsR = sc.parallelize(cells)
cellsR.take(5)

[[0, 42.0, -89.0],
 [1, 41.75, -89.0],
 [2, 41.5, -89.0],
 [3, 41.25, -89.0],
 [4, 41.0, -89.0]]

In [48]:
labeledCellsR = cellsR.map(lambda p: Row(cell_id=p[0],lat=p[1],lon=p[2]))
labeledCellsR.take(5)

[Row(cell_id=0, lat=42.0, lon=-89.0),
 Row(cell_id=1, lat=41.75, lon=-89.0),
 Row(cell_id=2, lat=41.5, lon=-89.0),
 Row(cell_id=3, lat=41.25, lon=-89.0),
 Row(cell_id=4, lat=41.0, lon=-89.0)]

In [49]:
labeledCellsDF = spark.createDataFrame(labeledCellsR)
labeledCellsDF.createOrReplaceTempView("cellsDF")
labeledCellsDF.show(5)

+-------+-----+-----+
|cell_id|  lat|  lon|
+-------+-----+-----+
|      0| 42.0|-89.0|
|      1|41.75|-89.0|
|      2| 41.5|-89.0|
|      3|41.25|-89.0|
|      4| 41.0|-89.0|
+-------+-----+-----+
only showing top 5 rows



In [50]:
cellFeatures = spark.sql("SELECT CAST(LEFT(a.time,4) AS INTEGER) AS YEAR, CAST(RIGHT(LEFT(a.time,7),2) AS INTEGER) AS MONTH, CAST(RIGHT(LEFT(a.time,10),2) AS INTEGER) AS DAY, CAST(RIGHT(LEFT(a.time,13),2) AS INTEGER) AS HOUR, a.level, CAST(b.cell_id AS INTEGER) AS cell_id,  a.t, a.u, a.v, a.w, a.r FROM allFeatures a  JOIN cellsDF b  ON a.latitude = b.lat AND a.longitude = b.lon ")
cellFeatures.createOrReplaceTempView("cellFeatures")
cellFeatures.show(5)

+----+-----+---+----+-----+-------+---------+---------+---------+---+----------+
|YEAR|MONTH|DAY|HOUR|level|cell_id|        t|        u|        v|  w|         r|
+----+-----+---+----+-----+-------+---------+---------+---------+---+----------+
|2022|    1|  1|   0|  100|    418|206.79155| 39.67998|1.8390284|0.0| 10.409676|
|2022|    1|  1|   0|  100|    291| 207.5068| 37.37952| 1.267134|0.0|  9.468033|
|2022|    1|  1|   0|  100|    374|207.14552|38.198967|2.0071042|0.0|10.1154175|
|2022|    1|  1|   0|  100|    164|207.69696|  40.6999|1.1601765|0.0|  8.037659|
|2022|    1|  1|   0|  100|    394|207.18208| 38.70517|1.9961901|0.0|  9.841621|
+----+-----+---+----+-----+-------+---------+---------+---------+---+----------+
only showing top 5 rows



Add labels (moved to a later step due to performance issues)

In [51]:
#storms = '/home/bcavna/Documents/ERA5/processed/processed_targets_20231118.csv'

In [52]:
custom_schema = StructType([
    StructField("cell",IntegerType(),False),
    StructField("YEAR",IntegerType(),False),
    StructField("MONTH",IntegerType(),False),
    StructField("DAY",IntegerType(),False)
])

In [53]:
#labelsDF = spark.read.schema(custom_schema).option("header","true").csv(storms)
#labelsDF.createOrReplaceTempView("labels")
#labelsDF.show(5)

In [54]:
#targetFeatures = spark.sql("SELECT a.YEAR,  a.MONTH,  a.DAY,  a.HOUR, a.cell_id, CASE WHEN b.YEAR IS NULL THEN 0 ELSE 1 END AS TARGET,  a.level,  a.q, a.t, a.u, a.v , a.w, a.r FROM cellFeatures a LEFT JOIN labels b ON a.cell_id = b.cell AND a.YEAR = b.YEAR AND a.MONTH = b.MONTH AND a.DAY = b.DAY ")
#targetFeatures.createOrReplaceTempView("targetFeatures")
#targetFeatures.show(5)

In [55]:
targetFeatures = spark.sql("SELECT a.YEAR,  a.MONTH,  a.DAY,  a.HOUR, a.cell_id,  a.level,  0 AS q, a.t, a.u, a.v , a.w, a.r FROM cellFeatures a WHERE a.DAY*1 > 30")
targetFeatures.createOrReplaceTempView("targetFeatures")
targetFeatures.show(5)

+----+-----+---+----+-------+-----+---+---------+---------+----------+---+---------+
|YEAR|MONTH|DAY|HOUR|cell_id|level|  q|        t|        u|         v|  w|        r|
+----+-----+---+----+-------+-----+---+---------+---------+----------+---+---------+
|2022|    1| 31|   0|    334|  100|  0|217.12411|16.703506| -10.30558|0.0|2.3212738|
|2022|    1| 31|   0|    271|  100|  0|216.67703| 16.82618|-10.507864|0.0|2.4696808|
|2022|    1| 31|   0|    145|  100|  0|216.02385|15.945942|-11.825727|0.0|2.7204437|
|2022|    1| 31|   0|    396|  100|  0| 217.0802| 17.06292|-10.480691|0.0|  2.33918|
|2022|    1| 31|   0|    100|  100|  0|216.10268| 15.75655|-10.947151|0.0| 2.692299|
+----+-----+---+----+-------+-----+---+---------+---------+----------+---+---------+
only showing top 5 rows



In [56]:
#checkTargets = spark.sql("SELECT  DISTINCT YEAR,  MONTH,  DAY, cell_id FROM targetFeatures WHERE TARGET = 1  ")
#checkTargets.show(10)

In [57]:
#ttl_rcrds = spark.sql("SELECT  COUNT(1) FROM targetFeatures")
#ttl_rcrds.show()

Get hourly files for smaller output

In [58]:
hourDF_00 = spark.sql("SELECT * FROM targetFeatures WHERE HOUR = 0")
hourPD_00 = hourDF_00.toPandas()
hourPD_00.head(2)

,YEAR,MONTH,DAY,HOUR,cell_id,level,q,t,u,v,w,r
0,2022,1,31,0,155,100,0,217.2097,17.609573,-8.515217,0.0,2.364769
1,2022,1,31,0,407,100,0,218.08136,22.219532,-6.0259175,0.0,2.0909805


In [59]:
hourDF_01 = spark.sql("SELECT * FROM targetFeatures WHERE HOUR = 1")
hourPD_01 = hourDF_01.toPandas()
hourPD_01.head(2)

,YEAR,MONTH,DAY,HOUR,cell_id,level,q,t,u,v,w,r
0,2022,1,31,1,105,100,0,217.15227,20.211555,-16.442026,0.0,2.3775635
1,2022,1,31,1,360,100,0,217.70071,20.932533,-9.553808,0.0,2.2214813


In [60]:
hourDF_02 = spark.sql("SELECT * FROM targetFeatures WHERE HOUR = 2")
hourPD_02 = hourDF_02.toPandas()
hourPD_02.head(2)

,YEAR,MONTH,DAY,HOUR,cell_id,level,q,t,u,v,w,r
0,2022,1,31,2,405,100,0,217.48111,21.696556,-9.141693,0.0,2.2854462
1,2022,1,31,2,289,100,0,216.45067,19.578815,-9.22472,0.0,2.5694733


In [61]:
hourDF_03 = spark.sql("SELECT * FROM targetFeatures WHERE HOUR = 3")
hourPD_03 = hourDF_03.toPandas()
hourPD_03.head(2)

,YEAR,MONTH,DAY,HOUR,cell_id,level,q,t,u,v,w,r
0,2022,1,31,3,104,100,0,212.95279,15.459549,-13.440979,0.0,3.9870605
1,2022,1,31,3,401,150,0,223.57144,17.953922,-18.931326,0.0,1.5408325


In [62]:
hourDF_04 = spark.sql("SELECT * FROM targetFeatures WHERE HOUR = 4")
hourPD_04 = hourDF_04.toPandas()
hourPD_04.head(2)

,YEAR,MONTH,DAY,HOUR,cell_id,level,q,t,u,v,w,r
0,2022,1,31,4,357,100,0,217.0937,18.3779,-14.34069,0.0,2.3698883
1,2022,1,31,4,400,100,0,217.2998,18.289661,-13.357952,0.0,2.3366241


In [63]:
hourDF_05 = spark.sql("SELECT * FROM targetFeatures WHERE HOUR = 5")
hourPD_05 = hourDF_05.toPandas()
hourPD_05.head(2)

,YEAR,MONTH,DAY,HOUR,cell_id,level,q,t,u,v,w,r
0,2022,1,31,5,373,100,0,214.78957,19.796185,-11.076975,0.0,3.188713
1,2022,1,31,5,426,125,0,220.62538,19.5164,-11.807613,0.0,1.8069534


In [64]:
hourDF_06 = spark.sql("SELECT * FROM targetFeatures WHERE HOUR = 6")
hourPD_06 = hourDF_06.toPandas()
hourPD_06.head(2)

,YEAR,MONTH,DAY,HOUR,cell_id,level,q,t,u,v,w,r
0,2022,1,31,6,276,100,0,216.95631,18.128246,-13.588919,0.0,2.3878021
1,2022,1,31,6,161,125,0,217.68832,19.996338,-15.561941,0.0,2.4466553


In [65]:
hourDF_07 = spark.sql("SELECT * FROM targetFeatures WHERE HOUR = 7")
hourPD_07 = hourDF_07.toPandas()
hourPD_07.head(2)

,YEAR,MONTH,DAY,HOUR,cell_id,level,q,t,u,v,w,r
0,2022,1,31,7,168,100,0,216.66125,21.031532,-13.611562,0.0,2.4287415
1,2022,1,31,7,242,100,0,215.66235,17.908726,-10.867144,0.0,2.7460327


In [66]:
hourDF_08 = spark.sql("SELECT * FROM targetFeatures WHERE HOUR = 8")
hourPD_08 = hourDF_08.toPandas()
hourPD_08.head(2)

,YEAR,MONTH,DAY,HOUR,cell_id,level,q,t,u,v,w,r
0,2022,1,31,8,150,100,0,216.24683,19.776814,-13.890835,0.0,2.5643616
1,2022,1,31,8,257,100,0,216.66238,19.744534,-12.912626,0.0,2.4236221


In [67]:
hourDF_09 = spark.sql("SELECT * FROM targetFeatures WHERE HOUR = 9")
hourPD_09 = hourDF_09.toPandas()
hourPD_09.head(2)

,YEAR,MONTH,DAY,HOUR,cell_id,level,q,t,u,v,w,r
0,2022,1,31,9,8,100,0,215.20963,18.479053,-16.31673,0.0,2.940506
1,2022,1,31,9,260,100,0,215.90785,19.208641,-11.917812,0.0,2.6795044


In [68]:
hourDF_10 = spark.sql("SELECT * FROM targetFeatures WHERE HOUR = 10")
hourPD_10 = hourDF_10.toPandas()
hourPD_10.head(2)

,YEAR,MONTH,DAY,HOUR,cell_id,level,q,t,u,v,w,r
0,2022,1,31,10,8,100,0,214.90443,20.297642,-16.786211,0.0,3.0428543
1,2022,1,31,10,29,125,0,217.04866,23.306381,-13.960276,0.0,2.7383575


In [69]:
hourDF_11 = spark.sql("SELECT * FROM targetFeatures WHERE HOUR = 11")
hourPD_11 = hourDF_11.toPandas()
hourPD_11.head(2)

,YEAR,MONTH,DAY,HOUR,cell_id,level,q,t,u,v,w,r
0,2022,1,31,11,32,100,0,214.2276,18.539314,-17.784044,0.0,3.3140945
1,2022,1,31,11,267,100,0,214.43031,16.940247,-14.037264,0.0,3.224533


In [70]:
hourDF_12 = spark.sql("SELECT * FROM targetFeatures WHERE HOUR = 12")
hourPD_12 = hourDF_12.toPandas()
hourPD_12.head(2)

,YEAR,MONTH,DAY,HOUR,cell_id,level,q,t,u,v,w,r
0,2022,1,31,12,168,100,0,216.56104,21.227383,-15.699312,0.0,2.4492111
1,2022,1,31,12,342,100,0,216.56891,20.020012,-14.369371,0.0,2.4492111


In [71]:
hourDF_13 = spark.sql("SELECT * FROM targetFeatures WHERE HOUR = 13")
hourPD_13 = hourDF_13.toPandas()
hourPD_13.head(2)

,YEAR,MONTH,DAY,HOUR,cell_id,level,q,t,u,v,w,r
0,2022,1,31,13,428,100,0,216.38197,18.748074,-13.6447735,0.0,2.5208588
1,2022,1,31,13,283,100,0,215.96754,15.933029,-16.146149,0.0,2.664154


In [72]:
hourDF_14 = spark.sql("SELECT * FROM targetFeatures WHERE HOUR = 14")
hourPD_14 = hourDF_14.toPandas()
hourPD_14.head(2)

,YEAR,MONTH,DAY,HOUR,cell_id,level,q,t,u,v,w,r
0,2022,1,31,14,394,100,0,215.92361,14.49107,-10.553151,0.0,2.6871796
1,2022,1,31,14,415,100,0,216.01822,14.693375,-10.201419,0.0,2.656479


In [73]:
hourDF_15 = spark.sql("SELECT * FROM targetFeatures WHERE HOUR = 15")
hourPD_15 = hourDF_15.toPandas()
hourPD_15.head(2)

,YEAR,MONTH,DAY,HOUR,cell_id,level,q,t,u,v,w,r
0,2022,1,31,15,439,100,0,215.75244,13.783005,-6.8516583,0.0,2.7434769
1,2022,1,31,15,188,100,0,215.1488,12.997459,-9.472291,0.0,2.966095


In [74]:
hourDF_16 = spark.sql("SELECT * FROM targetFeatures WHERE HOUR = 16")
hourPD_16 = hourDF_16.toPandas()
hourPD_16.head(2)

,YEAR,MONTH,DAY,HOUR,cell_id,level,q,t,u,v,w,r
0,2022,1,31,16,108,100,0,215.4878,16.931637,-12.965462,0.0,2.83815
1,2022,1,31,16,61,125,0,215.84366,16.918724,-16.380133,0.0,3.1349716


In [75]:
hourDF_17 = spark.sql("SELECT * FROM targetFeatures WHERE HOUR = 17")
hourPD_17 = hourDF_17.toPandas()
hourPD_17.head(2)

,YEAR,MONTH,DAY,HOUR,cell_id,level,q,t,u,v,w,r
0,2022,1,31,17,103,100,0,214.30531,13.957333,-7.774012,0.0,3.2577972
1,2022,1,31,17,439,100,0,214.98439,12.532589,-4.5842676,0.0,3.0147095


In [76]:
hourDF_18 = spark.sql("SELECT * FROM targetFeatures WHERE HOUR = 18")
hourPD_18 = hourDF_18.toPandas()
hourPD_18.head(2)

,YEAR,MONTH,DAY,HOUR,cell_id,level,q,t,u,v,w,r
0,2022,1,31,18,9,100,0,214.40892,14.960243,-5.6439934,0.0,3.227089
1,2022,1,31,18,127,125,0,218.15681,20.418163,-14.305969,0.0,2.4620056


In [77]:
hourDF_19 = spark.sql("SELECT * FROM targetFeatures WHERE HOUR = 19")
hourPD_19 = hourDF_19.toPandas()
hourPD_19.head(2)

,YEAR,MONTH,DAY,HOUR,cell_id,level,q,t,u,v,w,r
0,2022,1,31,19,299,100,0,216.67139,16.96392,-10.082162,0.0,2.4492111
1,2022,1,31,19,92,100,0,214.51816,17.164074,-4.7186203,0.0,3.188713


In [78]:
hourDF_20 = spark.sql("SELECT * FROM targetFeatures WHERE HOUR = 20")
hourPD_20 = hourDF_20.toPandas()
hourPD_20.head(2)

,YEAR,MONTH,DAY,HOUR,cell_id,level,q,t,u,v,w,r
0,2022,1,31,20,96,100,0,213.64313,18.989117,-3.2694225,0.0,3.5392685
1,2022,1,31,20,370,100,0,215.85942,14.844028,-6.755045,0.0,2.725563


In [79]:
hourDF_21 = spark.sql("SELECT * FROM targetFeatures WHERE HOUR = 21")
hourPD_21 = hourDF_21.toPandas()
hourPD_21.head(2)

,YEAR,MONTH,DAY,HOUR,cell_id,level,q,t,u,v,w,r
0,2022,1,31,21,156,100,0,214.01138,21.431839,-2.273099,0.0,3.398529
1,2022,1,31,21,55,100,0,213.20166,20.202946,-2.2142253,0.0,3.7183838


In [80]:
hourDF_22 = spark.sql("SELECT * FROM targetFeatures WHERE HOUR = 22")
hourPD_22 = hourDF_22.toPandas()
hourPD_22.head(2)

,YEAR,MONTH,DAY,HOUR,cell_id,level,q,t,u,v,w,r
0,2022,1,31,22,360,100,0,216.62521,19.710098,-9.537203,0.0,2.4696808
1,2022,1,31,22,165,100,0,212.60254,19.124706,-3.7464502,0.0,3.9947357


In [81]:
hourDF_23 = spark.sql("SELECT * FROM targetFeatures WHERE HOUR = 23")
hourPD_23 = hourDF_23.toPandas()
hourPD_23.head(2)

,YEAR,MONTH,DAY,HOUR,cell_id,level,q,t,u,v,w,r
0,2022,1,31,23,85,125,0,217.30994,22.299164,-7.4494524,0.0,2.5413284
1,2022,1,31,23,333,125,0,216.35269,18.283203,-13.244734,0.0,2.8790894


Split hourly data into daily to make the output even smaller (use pandas to help with varying number of days in each month)

In [82]:
daylist = list(range(1,32))
print(daylist)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]


In [83]:
def getBatch(daylist):
  for day in daylist:
      qstring = 'DAY == ' + str(day)
      #print(qstring)
      #print("")
      smaller_00 = hourPD_00.query(qstring)
      suffix = yearmo *10000 + day*100
      outpath = '/home/bcavna/Documents/ERA5/processed/smaller/ERA5_'+ str(suffix) + '.csv'
      print("outputting file to:",outpath)
      #print("")
      smaller_00.to_csv(outpath)
getBatch(daylist)

outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010100.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010200.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010300.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010400.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010500.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010600.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010700.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010800.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010900.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011000.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011100.csv
outputting file to: /home/bcavna/Documents/ERA5/proces

In [84]:
def getBatch(daylist):
  for day in daylist:
      qstring = 'DAY == ' + str(day)
      #print(qstring)
      #print("")
      smaller_01 = hourPD_01.query(qstring)
      suffix = yearmo *10000 + day*100 + 1
      outpath = '/home/bcavna/Documents/ERA5/processed/smaller/ERA5_'+ str(suffix) + '.csv'
      print("outputting file to:",outpath)
      #print("")
      smaller_01.to_csv(outpath)
getBatch(daylist)

outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010101.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010201.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010301.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010401.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010501.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010601.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010701.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010801.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010901.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011001.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011101.csv
outputting file to: /home/bcavna/Documents/ERA5/proces

In [85]:
def getBatch(daylist):
  for day in daylist:
      qstring = 'DAY == ' + str(day)
      #print(qstring)
      #print("")
      smaller_02 = hourPD_02.query(qstring)
      suffix = yearmo *10000 + day*100 + 2
      outpath = '/home/bcavna/Documents/ERA5/processed/smaller/ERA5_'+ str(suffix) + '.csv'
      print("outputting file to:",outpath)
      #print("")
      smaller_02.to_csv(outpath)
getBatch(daylist)

outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010102.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010202.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010302.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010402.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010502.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010602.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010702.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010802.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010902.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011002.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011102.csv
outputting file to: /home/bcavna/Documents/ERA5/proces

In [86]:
def getBatch(daylist):
  for day in daylist:
      qstring = 'DAY == ' + str(day)
      #print(qstring)
      #print("")
      smaller_03 = hourPD_03.query(qstring)
      suffix = yearmo *10000 + day*100 + 3
      outpath = '/home/bcavna/Documents/ERA5/processed/smaller/ERA5_'+ str(suffix) + '.csv'
      print("outputting file to:",outpath)
      #print("")
      smaller_03.to_csv(outpath)
getBatch(daylist)

outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010103.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010203.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010303.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010403.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010503.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010603.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010703.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010803.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010903.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011003.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011103.csv
outputting file to: /home/bcavna/Documents/ERA5/proces

In [87]:
def getBatch(daylist):
  for day in daylist:
      qstring = 'DAY == ' + str(day)
      #print(qstring)
      #print("")
      smaller_04 = hourPD_04.query(qstring)
      suffix = yearmo *10000 + day*100 + 4
      outpath = '/home/bcavna/Documents/ERA5/processed/smaller/ERA5_'+ str(suffix) + '.csv'
      print("outputting file to:",outpath)
      #print("")
      smaller_04.to_csv(outpath)
getBatch(daylist)

outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010104.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010204.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010304.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010404.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010504.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010604.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010704.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010804.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010904.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011004.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011104.csv
outputting file to: /home/bcavna/Documents/ERA5/proces

In [88]:
def getBatch(daylist):
  for day in daylist:
      qstring = 'DAY == ' + str(day)
      #print(qstring)
      #print("")
      smaller_05 = hourPD_05.query(qstring)
      suffix = yearmo *10000 + day*100 + 5
      outpath = '/home/bcavna/Documents/ERA5/processed/smaller/ERA5_'+ str(suffix) + '.csv'
      print("outputting file to:",outpath)
      #print("")
      smaller_05.to_csv(outpath)
getBatch(daylist)

outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010105.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010205.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010305.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010405.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010505.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010605.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010705.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010805.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010905.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011005.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011105.csv
outputting file to: /home/bcavna/Documents/ERA5/proces

In [89]:
def getBatch(daylist):
  for day in daylist:
      qstring = 'DAY == ' + str(day)
      #print(qstring)
      #print("")
      smaller_06 = hourPD_06.query(qstring)
      suffix = yearmo *10000 + day*100 + 6
      outpath = '/home/bcavna/Documents/ERA5/processed/smaller/ERA5_'+ str(suffix) + '.csv'
      print("outputting file to:",outpath)
      #print("")
      smaller_06.to_csv(outpath)
getBatch(daylist)

outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010106.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010206.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010306.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010406.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010506.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010606.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010706.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010806.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010906.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011006.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011106.csv
outputting file to: /home/bcavna/Documents/ERA5/proces

In [90]:
def getBatch(daylist):
  for day in daylist:
      qstring = 'DAY == ' + str(day)
      #print(qstring)
      #print("")
      smaller_07 = hourPD_07.query(qstring)
      suffix = yearmo *10000 + day*100 + 7
      outpath = '/home/bcavna/Documents/ERA5/processed/smaller/ERA5_'+ str(suffix) + '.csv'
      print("outputting file to:",outpath)
      #print("")
      smaller_07.to_csv(outpath)
getBatch(daylist)

outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010107.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010207.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010307.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010407.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010507.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010607.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010707.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010807.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010907.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011007.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011107.csv
outputting file to: /home/bcavna/Documents/ERA5/proces

In [91]:
def getBatch(daylist):
  for day in daylist:
      qstring = 'DAY == ' + str(day)
      #print(qstring)
      #print("")
      smaller_08 = hourPD_08.query(qstring)
      suffix = yearmo *10000 + day*100 + 8
      outpath = '/home/bcavna/Documents/ERA5/processed/smaller/ERA5_'+ str(suffix) + '.csv'
      print("outputting file to:",outpath)
      #print("")
      smaller_08.to_csv(outpath)
getBatch(daylist)

outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010108.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010208.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010308.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010408.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010508.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010608.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010708.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010808.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010908.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011008.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011108.csv
outputting file to: /home/bcavna/Documents/ERA5/proces

In [92]:
def getBatch(daylist):
  for day in daylist:
      qstring = 'DAY == ' + str(day)
      #print(qstring)
      #print("")
      smaller_09 = hourPD_09.query(qstring)
      suffix = yearmo *10000 + day*100 + 9
      outpath = '/home/bcavna/Documents/ERA5/processed/smaller/ERA5_'+ str(suffix) + '.csv'
      print("outputting file to:",outpath)
      #print("")
      smaller_09.to_csv(outpath)
getBatch(daylist)

outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010109.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010209.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010309.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010409.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010509.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010609.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010709.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010809.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010909.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011009.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011109.csv
outputting file to: /home/bcavna/Documents/ERA5/proces

In [93]:
def getBatch(daylist):
  for day in daylist:
      qstring = 'DAY == ' + str(day)
      #print(qstring)
      #print("")
      smaller_10 = hourPD_10.query(qstring)
      suffix = yearmo *10000 + day*100 + 10
      outpath = '/home/bcavna/Documents/ERA5/processed/smaller/ERA5_'+ str(suffix) + '.csv'
      print("outputting file to:",outpath)
      #print("")
      smaller_10.to_csv(outpath)
getBatch(daylist)

outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010110.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010210.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010310.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010410.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010510.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010610.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010710.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010810.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010910.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011010.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011110.csv
outputting file to: /home/bcavna/Documents/ERA5/proces

In [94]:
def getBatch(daylist):
  for day in daylist:
      qstring = 'DAY == ' + str(day)
      #print(qstring)
      #print("")
      smaller_11 = hourPD_11.query(qstring)
      suffix = yearmo *10000 + day*100 + 11
      outpath = '/home/bcavna/Documents/ERA5/processed/smaller/ERA5_'+ str(suffix) + '.csv'
      print("outputting file to:",outpath)
      #print("")
      smaller_11.to_csv(outpath)
getBatch(daylist)

outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010111.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010211.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010311.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010411.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010511.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010611.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010711.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010811.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010911.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011011.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011111.csv
outputting file to: /home/bcavna/Documents/ERA5/proces

In [95]:
def getBatch(daylist):
  for day in daylist:
      qstring = 'DAY == ' + str(day)
      #print(qstring)
      #print("")
      smaller_12 = hourPD_12.query(qstring)
      suffix = yearmo *10000 + day*100 + 12
      outpath = '/home/bcavna/Documents/ERA5/processed/smaller/ERA5_'+ str(suffix) + '.csv'
      print("outputting file to:",outpath)
      #print("")
      smaller_12.to_csv(outpath)
getBatch(daylist)

outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010112.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010212.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010312.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010412.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010512.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010612.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010712.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010812.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010912.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011012.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011112.csv
outputting file to: /home/bcavna/Documents/ERA5/proces

In [96]:
def getBatch(daylist):
  for day in daylist:
      qstring = 'DAY == ' + str(day)
      #print(qstring)
      #print("")
      smaller_13 = hourPD_13.query(qstring)
      suffix = yearmo *10000 + day*100 + 13
      outpath = '/home/bcavna/Documents/ERA5/processed/smaller/ERA5_'+ str(suffix) + '.csv'
      print("outputting file to:",outpath)
      #print("")
      smaller_13.to_csv(outpath)
getBatch(daylist)

outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010113.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010213.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010313.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010413.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010513.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010613.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010713.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010813.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010913.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011013.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011113.csv
outputting file to: /home/bcavna/Documents/ERA5/proces

In [97]:
def getBatch(daylist):
  for day in daylist:
      qstring = 'DAY == ' + str(day)
      #print(qstring)
      #print("")
      smaller_14 = hourPD_14.query(qstring)
      suffix = yearmo *10000 + day*100 + 14
      outpath = '/home/bcavna/Documents/ERA5/processed/smaller/ERA5_'+ str(suffix) + '.csv'
      print("outputting file to:",outpath)
      #print("")
      smaller_14.to_csv(outpath)
getBatch(daylist)

outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010114.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010214.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010314.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010414.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010514.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010614.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010714.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010814.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010914.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011014.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011114.csv
outputting file to: /home/bcavna/Documents/ERA5/proces

In [98]:
def getBatch(daylist):
  for day in daylist:
      qstring = 'DAY == ' + str(day)
      #print(qstring)
      #print("")
      smaller_15 = hourPD_15.query(qstring)
      suffix = yearmo *10000 + day*100 + 15
      outpath = '/home/bcavna/Documents/ERA5/processed/smaller/ERA5_'+ str(suffix) + '.csv'
      print("outputting file to:",outpath)
      #print("")
      smaller_15.to_csv(outpath)
getBatch(daylist)

outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010115.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010215.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010315.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010415.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010515.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010615.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010715.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010815.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010915.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011015.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011115.csv
outputting file to: /home/bcavna/Documents/ERA5/proces

In [99]:
def getBatch(daylist):
  for day in daylist:
      qstring = 'DAY == ' + str(day)
      #print(qstring)
      #print("")
      smaller_16 = hourPD_16.query(qstring)
      suffix = yearmo *10000 + day*100 + 16
      outpath = '/home/bcavna/Documents/ERA5/processed/smaller/ERA5_'+ str(suffix) + '.csv'
      print("outputting file to:",outpath)
      #print("")
      smaller_16.to_csv(outpath)
getBatch(daylist)

outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010116.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010216.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010316.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010416.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010516.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010616.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010716.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010816.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010916.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011016.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011116.csv
outputting file to: /home/bcavna/Documents/ERA5/proces

In [100]:
def getBatch(daylist):
  for day in daylist:
      qstring = 'DAY == ' + str(day)
      #print(qstring)
      #print("")
      smaller_17 = hourPD_17.query(qstring)
      suffix = yearmo *10000 + day*100 + 17
      outpath = '/home/bcavna/Documents/ERA5/processed/smaller/ERA5_'+ str(suffix) + '.csv'
      print("outputting file to:",outpath)
      #print("")
      smaller_17.to_csv(outpath)
getBatch(daylist)

outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010117.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010217.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010317.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010417.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010517.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010617.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010717.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010817.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010917.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011017.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011117.csv
outputting file to: /home/bcavna/Documents/ERA5/proces

In [101]:
def getBatch(daylist):
  for day in daylist:
      qstring = 'DAY == ' + str(day)
      #print(qstring)
      #print("")
      smaller_18 = hourPD_18.query(qstring)
      suffix = yearmo *10000 + day*100 + 18
      outpath = '/home/bcavna/Documents/ERA5/processed/smaller/ERA5_'+ str(suffix) + '.csv'
      print("outputting file to:",outpath)
      #print("")
      smaller_18.to_csv(outpath)
getBatch(daylist)

outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010118.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010218.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010318.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010418.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010518.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010618.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010718.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010818.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010918.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011018.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011118.csv
outputting file to: /home/bcavna/Documents/ERA5/proces

In [102]:
def getBatch(daylist):
  for day in daylist:
      qstring = 'DAY == ' + str(day)
      #print(qstring)
      #print("")
      smaller_19 = hourPD_19.query(qstring)
      suffix = yearmo *10000 + day*100 + 19
      outpath = '/home/bcavna/Documents/ERA5/processed/smaller/ERA5_'+ str(suffix) + '.csv'
      print("outputting file to:",outpath)
      #print("")
      smaller_19.to_csv(outpath)
getBatch(daylist)

outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010119.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010219.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010319.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010419.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010519.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010619.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010719.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010819.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010919.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011019.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011119.csv
outputting file to: /home/bcavna/Documents/ERA5/proces

In [103]:
def getBatch(daylist):
  for day in daylist:
      qstring = 'DAY == ' + str(day)
      #print(qstring)
      #print("")
      smaller_20 = hourPD_20.query(qstring)
      suffix = yearmo *10000 + day*100 + 20
      outpath = '/home/bcavna/Documents/ERA5/processed/smaller/ERA5_'+ str(suffix) + '.csv'
      print("outputting file to:",outpath)
      #print("")
      smaller_20.to_csv(outpath)
getBatch(daylist)

outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010120.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010220.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010320.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010420.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010520.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010620.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010720.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010820.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010920.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011020.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011120.csv
outputting file to: /home/bcavna/Documents/ERA5/proces

In [104]:
def getBatch(daylist):
  for day in daylist:
      qstring = 'DAY == ' + str(day)
      #print(qstring)
      #print("")
      smaller_21 = hourPD_21.query(qstring)
      suffix = yearmo *10000 + day*100 + 21
      outpath = '/home/bcavna/Documents/ERA5/processed/smaller/ERA5_'+ str(suffix) + '.csv'
      print("outputting file to:",outpath)
      #print("")
      smaller_21.to_csv(outpath)
getBatch(daylist)

outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010121.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010221.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010321.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010421.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010521.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010621.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010721.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010821.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010921.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011021.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011121.csv
outputting file to: /home/bcavna/Documents/ERA5/proces

In [105]:
def getBatch(daylist):
  for day in daylist:
      qstring = 'DAY == ' + str(day)
      #print(qstring)
      #print("")
      smaller_22 = hourPD_22.query(qstring)
      suffix = yearmo *10000 + day*100 + 22
      outpath = '/home/bcavna/Documents/ERA5/processed/smaller/ERA5_'+ str(suffix) + '.csv'
      print("outputting file to:",outpath)
      #print("")
      smaller_22.to_csv(outpath)
getBatch(daylist)

outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010122.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010222.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010322.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010422.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010522.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010622.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010722.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010822.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010922.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011022.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011122.csv
outputting file to: /home/bcavna/Documents/ERA5/proces

In [106]:
def getBatch(daylist):
  for day in daylist:
      qstring = 'DAY == ' + str(day)
      #print(qstring)
      #print("")
      smaller_23 = hourPD_23.query(qstring)
      suffix = yearmo *10000 + day*100 + 23
      outpath = '/home/bcavna/Documents/ERA5/processed/smaller/ERA5_'+ str(suffix) + '.csv'
      print("outputting file to:",outpath)
      #print("")
      smaller_23.to_csv(outpath)
getBatch(daylist)

outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010123.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010223.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010323.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010423.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010523.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010623.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010723.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010823.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022010923.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011023.csv
outputting file to: /home/bcavna/Documents/ERA5/processed/smaller/ERA5_2022011123.csv
outputting file to: /home/bcavna/Documents/ERA5/proces